<a href="https://colab.research.google.com/github/iserveradmin/GDG/blob/main/GDG-New-Delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -q --upgrade sentence-transformers faiss-cpu
%pip install -q "numpy==2.0.2" "pandas==2.2.2" "scikit-learn>=1.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 93.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re

# Create a diverse list of Q&A about finding doctors in Dwarka, Delhi, and other Indian cities
qa_data = [
    {"question": "I need a doctor for sugar patient in Dwarka tomorrow",
     "Answer": "You can consult Dr. A (Endocrinologist, Dwarka) or Dr. B (Diabetologist, Dwarka Sector-6) for diabetes care."},

    {"question": "Best dentist available in Dwarka today?",
     "Answer": "Dr. Mehta Dental Clinic in Dwarka Sector-12 is available today for dental care."},

    {"question": "Who is the best pediatrician in South Delhi?",
     "Answer": "Dr. Sharma at Apollo Cradle, South Delhi, is a highly recommended pediatrician."},

    {"question": "Where can I find a gynecologist in Saket, Delhi?",
     "Answer": "You can consult Dr. Priya at Max Super Speciality Hospital, Saket."},

    {"question": "I have stomach pain, which doctor should I consult?",
     "Answer": "You should consult a gastroenterologist for stomach pain issues."},

    {"question": "Need orthopedic doctor in Noida Sector 18",
     "Answer": "Dr. R.K. Verma, Orthopedic Specialist, is available in Noida Sector 18."},

    {"question": "Suggest a cardiologist in Gurgaon near Cyber City",
     "Answer": "You may consult Dr. Kapoor, Senior Cardiologist at Fortis Hospital, Gurgaon."},

    {"question": "Looking for a skin specialist in Dwarka",
     "Answer": "Dr. Pooja, Dermatologist, runs a clinic in Dwarka Sector-10."},

    {"question": "Who is a good neurologist in AIIMS Delhi?",
     "Answer": "AIIMS Delhi has Dr. Anil Kumar, an experienced neurologist for brain and nerve-related issues."},

    {"question": "ENT doctor available in Rohini today?",
     "Answer": "Dr. Sinha, ENT Specialist, is available at Rohini Sector-9 clinic today."},
]

# Expand the dataset programmatically to reach ~100 entries
cities = ["Dwarka", "Saket", "Noida", "Gurgaon", "Rohini", "South Delhi", "East Delhi", "West Delhi", "Mumbai", "Bangalore"]
specialties = [
    ("diabetes", "Endocrinologist/Diabetologist"),
    ("heart issues", "Cardiologist"),
    ("stomach pain", "Gastroenterologist"),
    ("skin problems", "Dermatologist"),
    ("children health", "Pediatrician"),
    ("bones pain", "Orthopedic"),
    ("ear pain", "ENT Specialist"),
    ("mental health", "Psychiatrist"),
    ("eyes checkup", "Ophthalmologist"),
    ("fever and cold", "General Physician"),
]

for city in cities:
    for condition, doctor in specialties:
        qa_data.append({
            "question": f"I need a doctor for {condition} in {city}",
            "Answer": f"You should consult a {doctor} in {city} for {condition} treatment."
        })

# Ensure we have at least ~100 entries
qa_data = qa_data[:100]

# Create DataFrame
df = pd.DataFrame(qa_data)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply preprocessing to questions
df['question'] = df['question'].apply(preprocess_text)

display(df.head())

,question,Answer
0,i need a doctor for sugar patient in dwarka to...,"You can consult Dr. A (Endocrinologist, Dwarka..."
1,best dentist available in dwarka today,Dr. Mehta Dental Clinic in Dwarka Sector-12 is...
2,who is the best pediatrician in south delhi,"Dr. Sharma at Apollo Cradle, South Delhi, is a..."
3,where can i find a gynecologist in saket delhi,You can consult Dr. Priya at Max Super Special...
4,i have stomach pain which doctor should i consult,You should consult a gastroenterologist for st...


In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re

# Load a pre-trained sentence transformer model
# You can try different models here, e.g., 'all-mpnet-base-v2' for potentially better results
model = SentenceTransformer('all-MiniLM-L6-v2')

# Preprocessing function (duplicate for clarity, ideally in a shared utility)
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Generate embeddings for the preprocessed questions
question_embeddings = model.encode(df['question'].tolist())

# Get the dimensionality of the embeddings
dimension = question_embeddings.shape[1]

# Create a FAISS index (using a simple IndexFlatL2 for demonstration)
index = faiss.IndexFlatL2(dimension)

# Add the embeddings to the index
index.add(question_embeddings)

print(f"Number of embeddings in the index: {index.ntotal}")

Number of embeddings in the index: 100


In [6]:
# Define a query question
query_question = "i have cold. suggest me a doctor"

# Generate embedding for the query question
query_embedding = model.encode([preprocess_text(query_question)]) # Preprocess the query

# Perform a similarity search (e.g., find the 3 most similar questions)
k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Retrieve the corresponding answers from the original DataFrame
print(f"Query: {query_question}")
print("\nBest matching questions and their answers:")
for i in range(k):
    print(f"Answer: {df['Answer'].iloc[indices[0][i]]}")
    print(f"Distance: {distances[0][i]:.4f}")
    print("-" * 20)

Query: i have cold. suggest me a doctor

Best matching questions and their answers:
Answer: You should consult a General Physician in Saket for fever and cold treatment.
Distance: 0.8673
--------------------
Answer: You should consult a General Physician in Rohini for fever and cold treatment.
Distance: 1.0182
--------------------
Answer: You should consult a General Physician in South Delhi for fever and cold treatment.
Distance: 1.0579
--------------------


In [7]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

# Ensure the GOOGLE_API_KEY is set up in Colab secrets
try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Error setting up Gemini API key: {e}")
    print("Please ensure you have set up the GOOGLE_API_KEY correctly in Colab secrets.")
    # Exit the cell execution if API key setup fails
    raise

# Initialize the Gemini API
# You can choose a different model if needed, e.g., 'gemini-pro'
try:
    gemini_model = genai.GenerativeModel('gemini-2.5-flash')
except Exception as e:
    print(f"Error initializing Gemini model: {e}")
    # Exit the cell execution if model initialization fails
    raise


# Get the original query and the top answers from the previous search
# Assuming 'query_question', 'indices', and 'k' are available from the previous search cell (3a6d003c)
query = query_question
retrieved_answers = [df['Answer'].iloc[indices[0][i]] for i in range(k)]

# Craft a prompt for the LLM, emphasizing the use of only the provided context
prompt = f"""
Based ONLY on the following query and retrieved information, provide a helpful and conversational response.
Do NOT include any information not present in the provided context.

Query: "{query}"

Retrieved Information:
{'- ' + '\\n- '.join(retrieved_answers)}

Please synthesize this information and provide a clear answer to the user's query, strictly using the provided context.
"""

# Generate a response using the LLM
try:
    response = gemini_model.generate_content(prompt)
    print("LLM's Response (based only on provided context):")
    print(response.text)
except Exception as e:
    print(f"An error occurred while generating the LLM response: {e}")
    print("Please ensure you have set up the GOOGLE_API_KEY correctly in Colab secrets and have run the cells to initialize the model.")

LLM's Response (based only on provided context):
It sounds like you have a cold. For this, you should consult a General Physician. You can consult one in Saket, Rohini, or South Delhi.


In [8]:
import gradio as gr

# take input from user as question
#initialized the prompt and take as input
#submit query to gemeni
#deliver response

def search_ui(query_ui):
  # Generate embedding for the query question
  query_embedding_ui = model.encode([preprocess_text(query_ui)]) # Preprocess the query

# Perform a similarity search (e.g., find the 3 most similar questions)
  k = 3  # Number of nearest neighbors to retrieve
  distances, indices = index.search(query_embedding_ui, k)

# Retrieve the corresponding answers from the original DataFrame
  print(f"Query: {query_ui}")
  print("\nBest matching questions and their answers:")
  retrieved_answers = [df['Answer'].iloc[indices[0][i]] for i in range(k)]
  prompt_ui = f"""
  Based ONLY on the following query and retrieved information, provide a helpful and conversational response.Do NOT include any information not present in the provided context.
  Query: "{query_ui}"
  Retrieved Information:"{'- ' + '\\n- '.join(retrieved_answers)}"
  Please synthesize this information and provide a clear answer to the user's query, strictly using the provided context.
  """
  result = gemini_model.generate_content(prompt_ui)
  return result.text

gr.Interface(fn=search_ui, inputs="text", outputs="text").launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://265afb17a54d33cc3a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://265afb17a54d33cc3a.gradio.live
